In [13]:
import sys
import rospy
import rospkg
import moveit_commander
import tf
import os 
import numpy as np 
import copy
import time

from geometry_msgs.msg import (
    PoseStamped,
    Pose,
    Point,
    Quaternion,
)
from gazebo_msgs.srv import (SpawnModel,DeleteModel)
from geometry_msgs.msg import (PoseStamped,Pose,Point,Quaternion)

def spawn_gazebo_model(model_path, model_name, model_pose, reference_frame="world"):
  """
  Spawn model in gazebo
  """
  model_xml = ''
  with open(model_path, "r") as model_file:
    model_xml = model_file.read().replace('\n', '')
  rospy.wait_for_service('/gazebo/spawn_sdf_model')
  try:
    spawn_sdf = rospy.ServiceProxy('/gazebo/spawn_sdf_model', SpawnModel)
    resp_sdf= spawn_sdf(model_name, model_xml, "/", model_pose, reference_frame)
  except rospy.ServiceException:
    rospy.logerr("Spawn SDF service call failed")

def delete_gazebo_model(models):
  """
  Delete model in gazebo
  """
  try:
    delete_model = rospy.ServiceProxy('/gazebo/delete_model', DeleteModel)
    for a_model in models:
      resp_delete = delete_model(a_model)
  except rospy.ServiceException:
    rospy.loginfo("Delete Model service call failed")

print("Packages Loaded.")
rospy.init_node("object_spawn")

Packages Loaded.


### Set Moveit 

In [14]:
moveit_commander.roscpp_initialize(sys.argv)
robot         = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()

### Set Parameters 

In [15]:
# Parameters 
table_height    = 0.79
# Cylinder 
cylinder_height = 0.2
cylinder_radius = 0.03
cylinder_pos_x  = 0.97 
cylinder_pos_y  = 0 
cylinder_pos_z  = table_height+cylinder_height/2.
# Box 
box_height      = 0.06
box_x           = 0.06 
box_y           = 0.06
box_pos_x       = 0.97 
box_pos_y       = 0 
box_pos_z       = table_height+box_height/2.

In [16]:
""" RViz """
# Cylinder 
cylinder_name = "cylinder"
cylinder_pos  = PoseStamped()
cylinder_pos.header.frame_id    = robot.get_planning_frame()
cylinder_pos.pose.position.x    = cylinder_pos_x
cylinder_pos.pose.position.y    = cylinder_pos_y
cylinder_pos.pose.position.z    = cylinder_pos_z
cylinder_pos.pose.orientation.w = 1.0  
# Box 
box_name = "box"
box_pos  = PoseStamped()
box_pos.header.frame_id    = robot.get_planning_frame()
box_pos.pose.position.x    = box_pos_x
box_pos.pose.position.y    = box_pos_y
box_pos.pose.position.z    = box_pos_z
box_pos.pose.orientation.w = 1.0  

""" GAZEBO """
rospack = rospkg.RosPack()
pack_path = rospack.get_path('ir_gazebo')
object_path = pack_path+os.sep+'urdf'+os.sep+'object'+os.sep+'cylinder.sdf'
object_name = cylinder_name
object_pose = Pose(position=Point(x=cylinder_pos_x , y=cylinder_pos_y, z=cylinder_pos_z))


### Spawn Object 

In [19]:
# If you want to add a cylinder on RViz 
scene.add_cylinder(name=cylinder_name, height=cylinder_height, radius=cylinder_radius, pose=cylinder_pos)

# If you want to add a box on RViz 
# scene.add_box(box_name, box_pos, (box_x, box_y, box_height))

# If you want to add a cylinder on GAZEBO 
spawn_gazebo_model(object_path, object_name, object_pose)

### Delete Object 

In [18]:
delete_gazebo_model([object_name])
scene.remove_world_object(object_name)
# scene.remove_world_object("box")
print("Delete %s"%object_name)

Delete cylinder
